# CycleGAN for Monet Paintings Dataset
## Kaggle Competition: GAN Getting Started

This notebook implements a CycleGAN to translate real-world photos into Monet-style paintings.

**Objective:** Convert real-world photographs into Monet-style paintings using Generative Adversarial Networks (GANs), specifically a CycleGAN architecture.

**Dataset:**
- 300 Monet paintings
- 7038 real-world photos

The goal is to train a model that learns artistic transformations from Monet’s works and applies them to photographs.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, LeakyReLU, ReLU, Input
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import shutil
from kaggle_datasets import KaggleDatasets

## Load and Prepare Data

In [ ]:
# Define the data path
GCS_PATH = KaggleDatasets().get_gcs_path()

# Get the list of Monet and Photo image files
MONET_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/monet_jpg/*.jpg'))
PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/photo_jpg/*.jpg'))

# Print the number of files found
print('Monet jpg Files:', len(MONET_FILENAMES))
print('Photo jpg Files:', len(PHOTO_FILENAMES))

## Exploratory Data Analysis (EDA)

In [ ]:
# Load and display an example Monet image & Photo image
example_monet_path = MONET_FILENAMES[0]
monet_image = tf.io.read_file(example_monet_path)
monet_image = tf.image.decode_jpeg(monet_image, channels=3)

example_photo_path = PHOTO_FILENAMES[0]
photo_image = tf.io.read_file(example_photo_path)
photo_image = tf.image.decode_jpeg(photo_image, channels=3)

# Display the images side by side
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(monet_image.numpy())
plt.title("Example Monet Image")
plt.subplot(1, 2, 2)
plt.imshow(photo_image.numpy())
plt.title("Example Photo Image")
plt.show()

## Data Preprocessing

In [ ]:
IMAGE_SIZE = [256, 256]
BATCH_SIZE = 1

def decode_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

monet_ds = tf.data.Dataset.from_tensor_slices(MONET_FILENAMES).map(decode_image).batch(BATCH_SIZE)
photo_ds = tf.data.Dataset.from_tensor_slices(PHOTO_FILENAMES).map(decode_image).batch(BATCH_SIZE)

## Model Definition

In [ ]:
def build_generator():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(32, (7, 7), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Conv2DTranspose(3, (7, 7), padding='same', activation='tanh')(x)
    return keras.Model(inputs, x, name="Generator")

def build_discriminator():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(32, (4, 4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(1, (4, 4), padding='same')(x)
    return keras.Model(inputs, x, name="Discriminator")

## Training the CycleGAN

In [ ]:
cycle_gan = CycleGAN()
cycle_gan.train(photo_ds, monet_ds, epochs=20)

## Results and Generated Images

In [ ]:
!mkdir ../images

i = 1
for img in photo_ds:
    prediction = cycle_gan.models["generator_G"](img, training=False)[0].numpy()
    prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
    im = PIL.Image.fromarray(prediction)
    im.save(f"../images/{i}.jpg")
    i += 1

shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")

## Conclusion

- We trained a CycleGAN to transform real-world images into Monet-style paintings.
- The model successfully generated Monet-style paintings but requires more training for fine details.
- Future work includes longer training, better hyperparameter tuning, and possibly using perceptual loss for improved texture reproduction.